# Maxarcat tutorial

Maxarcat is a Python package that provides simple methods for searching the
[Maxar Catalog (https://doc.content.maxar.com)](https://doc.content.maxar.com).
Read the documentation at the Maxar Catalog website to under the basics of using the catalog.

The package is based on a client generated by [Swagger codegen](https://github.com/swagger-api/swagger-codegen).
On top of this the `Catalog` class provides a simple wrapper of the methods generated by Swagger.

This notebook shows a few examples of using the `Catalog` class to query the Maxar Catalog,
and visualizes the results using [geopandas](https://geopandas.org), [shapely](https://github.com/Toblerity/Shapely),
and [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet).

Begin by importing reqiured modules, including mararcat's `Catalog` class:

In [1]:
import logging
import sys
from datetime import datetime
from pprint import pprint

import geopandas
import ipyleaflet
import shapely.geometry

from maxarcat import Catalog

You will need GBDX credentials to use the Maxar catalog, which in turn are used to generate
a bearer token allowing API access.  You can construct a `Catalog` object
by passing in a GBDX token if you have obtained one yourself.  Or call the `Catalog.connect` method
to be prompted for your GBDX credentials, have a new GBDX token generated, and get a new `Catalog`
object returned.

A `Catalog` object logs helpful information at the INFO level.  Log messages print query parameters,
elapsed time, HTTP status, and the value of the returned X-Maxar-RequestId HTTP header,
which can be sent to Maxar when diagnosing problems with requests.  For these examples we initialize the
logging level to INFO so we see these messages.

In [ ]:
logging.basicConfig(level=logging.INFO)
catalog = Catalog.connect()

# Working with collections

Request a collection by calling `get_collection` with its ID.
Here is an example of getting the `wv02` catalog:

In [3]:
coll = catalog.get_collection('wv02')

INFO:maxar.catalog:Get collection wv02
INFO:maxar.catalog:Elapsed seconds: 3.38
INFO:maxar.catalog:HTTP Status: 200
INFO:maxar.catalog:Request ID: 7a71db3f-c45b-4e96-9640-6cd19969db88


In [4]:
coll.id, coll.description

('wv02', 'WorldView-2 Imagery')

Call `get_collections` to request all collections in the catalog.
This results in a large number of collections because every Maxar Vivid mosaic is treated as a
separate collection, and there are hundreds of Vivid mosaics in the catalog:

In [5]:
collections = catalog.get_collections()
len(collections.collections)

INFO:maxar.catalog:Get all collections
INFO:maxar.catalog:Elapsed seconds: 5.22
INFO:maxar.catalog:HTTP Status: 200
INFO:maxar.catalog:Request ID: 733d09d6-7978-4ed7-9afa-bbf509539111


589

# Working with items

An item can be fetched by calling `get_item` with its image identifier:

In [6]:
item = catalog.get_item('103001008C904200')

INFO:maxar.catalog:Get item 103001008C904200
INFO:maxar.catalog:Elapsed seconds: 0.45
INFO:maxar.catalog:HTTP Status: 200
INFO:maxar.catalog:Request ID: d24fecf3-179c-489b-8b73-31e21d31659c


In [7]:
pprint(item.properties)

{'acquisition_rev_number': 48934,
 'associations': [],
 'collect_time_end': '2019-02-04T16:50:55.010386Z',
 'collect_time_start': '2019-02-04T16:50:50.760842Z',
 'constellation': 'digitalglobe',
 'datetime': '2019-02-04T16:50:50.760842Z',
 'eo:bands': [{'center_wavelength': 950, 'name': 'nir2'},
              {'center_wavelength': 895, 'name': 'nir1'},
              {'center_wavelength': 725, 'name': 'red_edge'},
              {'center_wavelength': 660, 'name': 'red'},
              {'center_wavelength': 605, 'name': 'yellow'},
              {'center_wavelength': 545, 'name': 'green'},
              {'center_wavelength': 480, 'name': 'blue'},
              {'center_wavelength': 425, 'name': 'coastal'},
              {'center_wavelength': 625, 'name': 'pan'}],
 'eo:cloud_cover': 7.26021196531,
 'gsd': 0.56872135,
 'instruments': ['VNIR'],
 'multi_resolution_avg': 2.2836306,
 'multi_resolution_end': 2.3023689,
 'multi_resolution_max': 2.3023689,
 'multi_resolution_min': 2.2583313,
 'mult

# Searching

Use the `search` method to search the catalog using a variety of filters.

The catalog contains many collections of geospatial products, not just satellite images.
To search on just satellite images you should specify the following set of collections when searching.
Otherwise the search responses can contain items like DEM's, mosaics, and other products:

In [8]:
collections = ['ge01', 'wv01', 'wv02', 'wv03-vnir', 'wv04']

Here is an example of searching for images inside a bounding box:

In [9]:
items = catalog.search(bbox=[-105, 40, -104, 41], collections=collections)

INFO:maxar.catalog:Search collections: ['ge01', 'wv01', 'wv02', 'wv03-vnir', 'wv04']
INFO:maxar.catalog:Search bbox: [-105, 40, -104, 41]
INFO:maxar.catalog:Elapsed seconds: 1.21
INFO:maxar.catalog:HTTP Status: 200
INFO:maxar.catalog:Request ID: 15b1c238-7ed2-4195-bea4-17e21b5ab114


To view the results on a map we create a small helper function `map_features`.
It creates a PyLeaflet Map widget displaying a list of STAC items.

In [10]:
def map_features(features, zoom_level):
    """
    Construct and return an ipyleaflet Map displaying a layer for the given features.
    :param features: Sequence of Item objects (Swagger-geneated Item models) returned from a query
    :param zoom_level: Zoom level to set map at
    """
    shapes = [shapely.geometry.shape(feature.geometry) for feature in features]
    center = shapely.geometry.GeometryCollection(shapes).centroid.coords[0]
    map = ipyleaflet.Map(
        center=(center[1], center[0]),  # Leaflet requires coordinates in order latitude, longitude
        zoom=zoom_level,
        basemap=ipyleaflet.basemaps.OpenStreetMap.Mapnik)
    gdf = geopandas.GeoDataFrame(geometry=shapes)
    map.add_layer(ipyleaflet.GeoData(geo_dataframe=gdf))
    return map

In [11]:
map_features(items.features, 6)

Map(center=[40.42371355644256, -104.6863953109404], controls=(ZoomControl(options=['position', 'zoom_in_text',…

Here is a more complex search that finds WorldView-2 images over Australia during the first two days of 2020:

In [12]:
items = catalog.search(
    bbox=[110, -44, 155, -9],
    collections=['wv02'],
    start_datetime=datetime(2020, 1, 1),
    end_datetime=datetime(2020, 1, 3))

INFO:maxar.catalog:Search collections: ['wv02']
INFO:maxar.catalog:Search bbox: [110, -44, 155, -9]
INFO:maxar.catalog:Search time range: 2020-01-01T00:00:00.000000Z/2020-01-03T00:00:00.000000Z
INFO:maxar.catalog:Elapsed seconds: 0.70
INFO:maxar.catalog:HTTP Status: 200
INFO:maxar.catalog:Request ID: ce366f5d-9daf-4088-b1cc-195c0782bfba


In [13]:
map_features(items.features, 4)

Map(center=[-27.148485873158645, 138.21872886903537], controls=(ZoomControl(options=['position', 'zoom_in_text…

The above examples return a small number of items.  The service has a limit on the number of items
it will return for any search, generally 100.  The caller can use the search parameters `page` and `limit`
to repeatedly page through search results, stopping when no more results are returned for a page.

Alternately you can call the `query` method to fetch results iteratively.  `query` accepts
the same search parameters as `search` but returns a Python generator.  The generator
returns one item from the result each time it is called, querying the next page of results
when necessary.

Here is an example of searching over Australia again, but this time using all collections:

In [14]:
collections = ['ge01', 'wv01', 'wv02', 'wv03-vnir', 'wv04']
items = catalog.query(
    bbox=[110, -44, 155, -9],
    collections=collections,
    start_datetime=datetime(2020, 1, 1),
    end_datetime=datetime(2020, 1, 3))

In [15]:
ids = [item.id for item in items]    # Force all pages to be queried

INFO:maxar.catalog:Query page 1
INFO:maxar.catalog:Search collections: ['ge01', 'wv01', 'wv02', 'wv03-vnir', 'wv04']
INFO:maxar.catalog:Search bbox: [110, -44, 155, -9]
INFO:maxar.catalog:Search time range: 2020-01-01T00:00:00.000000Z/2020-01-03T00:00:00.000000Z
INFO:maxar.catalog:Search page: 1
INFO:maxar.catalog:Elapsed seconds: 1.14
INFO:maxar.catalog:HTTP Status: 200
INFO:maxar.catalog:Request ID: 816c6bf8-91a7-485c-b175-cd7b573d05f3
INFO:maxar.catalog:Query page 2
INFO:maxar.catalog:Search collections: ['ge01', 'wv01', 'wv02', 'wv03-vnir', 'wv04']
INFO:maxar.catalog:Search bbox: [110, -44, 155, -9]
INFO:maxar.catalog:Search time range: 2020-01-01T00:00:00.000000Z/2020-01-03T00:00:00.000000Z
INFO:maxar.catalog:Search page: 2
INFO:maxar.catalog:Elapsed seconds: 0.85
INFO:maxar.catalog:HTTP Status: 200
INFO:maxar.catalog:Request ID: 19dd7b56-932e-4150-a367-d8ab36332715
INFO:maxar.catalog:Query page 3
INFO:maxar.catalog:Search collections: ['ge01', 'wv01', 'wv02', 'wv03-vnir', 'wv04']

# Working with assets

Items in the Maxar Catalog are GeoJSON features that adhere to the
[STAC specification](https://stacspec.org).  Among their properties can
be assets with URL's.

Catalog items for Maxar satellite images generally contain three assets:
* browse.  URL to GeoTIFF browse image
* cloud-cover.  URL to GeoJSON feature with cloud polygon (or multipolygon)
* sample-point-set.  URL to GeoJSON feature of image sample points

The following example shows how to fetch the cloud cover polygon of a Maxar
satellite image.  First request an image by its ID and draw its footprint on a map:

In [16]:
item = catalog.get_item('105001001D5FFA00')

INFO:maxar.catalog:Get item 105001001D5FFA00
INFO:maxar.catalog:Elapsed seconds: 0.15
INFO:maxar.catalog:HTTP Status: 200
INFO:maxar.catalog:Request ID: c648bcdd-8708-4f30-b0fe-e05911d11f01


In [17]:
map = map_features([item], 10)
map

Map(center=[40.18295493509207, -104.98126841259653], controls=(ZoomControl(options=['position', 'zoom_in_text'…

An Item's `asset` property is a dictionary keyed on asset names, and each value is itself
a dictionary of asset properties.  See the
[STAC item specification](https://github.com/radiantearth/stac-spec/blob/master/item-spec/item-spec.md#asset-object)
for details.

In [18]:
cloud_href = item.assets['cloud-cover']['href']
cloud_href

'https://beta-api.content.satcloud.us/catalog/collections/ge01/items/105001001D5FFA00/assets/collections/ge01/assets/cloud-cover/105001001D5FFA00.cloud.json'

Call `get_url_json` to perform a GET on an asset URL, parse the respose as JSON, and return it:

In [19]:
cloud = catalog.get_url_json(cloud_href)

INFO:root:GET https://beta-api.content.satcloud.us/catalog/collections/ge01/items/105001001D5FFA00/assets/collections/ge01/assets/cloud-cover/105001001D5FFA00.cloud.json
INFO:maxar.catalog:Elapsed seconds: 1.71
INFO:maxar.catalog:HTTP Status: 200


Display the cloud polygons on the map above.
Generate a GeoDataFrame with one row for the cloud shape and add as a layer to the map:

In [20]:
gdf = geopandas.GeoDataFrame(geometry=[shapely.geometry.shape(cloud['geometry'])])
map.add_layer(ipyleaflet.GeoData(geo_dataframe=gdf))